In [1]:
import requests
import polars as pl
import pandas as pd
from pathlib import Path
import geopandas as gpd
from tqdm.notebook import tqdm
from bs4 import BeautifulSoup

from datetime import datetime

In [2]:
base = Path.home() / "Local_Workspace" / "Datasets" / "ARPA" / "TRENTINO" / "bolzano"

## API Bolzano
Stazioni recuperabili tramite API OpenData. Solo dal 2014.

In [10]:
sensors = pl.read_json(base / "api" / "sensors.json").with_columns(pl.col("SCODE").str.to_uppercase().str.strip_chars())
sensors.write_csv(base / "api" / "sensors.csv")
stations = gpd.read_file(base / "api" / "stations.geojson")
stations["SCODE"] = stations["SCODE"].str.upper().str.strip()

In [5]:
def query_payload(station_code, from_date: datetime, to_date: datetime):
    return {
        "station_code": station_code,
        "sensor_code": "LT",
        "date_from": from_date.strftime(r"%Y%m%d"),
        "date_to": to_date.strftime(r"%Y%m%d"),
        "output_format": "CSV",
    }

In [6]:
stat_ids = sensors.filter(pl.col("TYPE").eq("LT"))["SCODE"].to_list()

In [23]:
from time import sleep
from random import uniform


def file_path(station_id, year):
    path = base / "fragments" / f"{station_id}/{year}.csv"
    if not path.parent.exists():
        path.parent.mkdir(parents=True)
    return path


def get_year_data(station_id, year):
    try:
        r = requests.get(
            "http://daten.buergernetz.bz.it/services/meteo/v1/timeseries",
            params=query_payload(
                station_id, datetime(year, 1, 1), datetime(year + 1, 1, 1)
            ),
        )
        if r.status_code != 200:
            sleep(uniform(3, 5))
            raise
        return r.text
    except:
        print(f"There was an error: {r.status_code}. Continuing...")
        return False


def get_station_data(station_id, bar: tqdm):
    for year in range(2000, 2023):
        path = file_path(station_id, year)
        if path.exists():
            bar.update()
            continue
        data = get_year_data(station_id, year)
        if not data:
            bar.update()
            continue
        if data.strip() == "":
            bar.update()
            continue
        with open(path, "wt") as file:
            file.write(data)
        bar.update()
        sleep(uniform(0.5, 1.5))

    bar.reset(total=2023 - 2000)

In [ ]:
for station_id in tqdm(stat_ids):
    with tqdm(total=2023 - 2000, leave=True) as bar:
        get_station_data(station_id, bar)

In [16]:
for file in (base / "api" / "raw").glob("**/*.csv"):
    station_id = file.parent.stem
    file_path = base / "api" / "raw_parquet" / f"{station_id}" / f"{file.stem}.parquet"
    if not file_path.parent.exists():
        file_path.parent.mkdir(parents=True)
    pl.read_csv(file, try_parse_dates=False).with_columns(
        pl.col("DATE").str.to_datetime(format=r"%Y-%m-%dT%H:%M:%S%Z"),
        pl.lit(station_id).str.to_uppercase().str.strip_chars().alias("original_id"),
    ).write_parquet(file_path)

## XLSX Data
Sempre da OpenData ma in formato tabelle excel

In [14]:
page = (base / "xlsx" / "xlsx_datastore.html").read_text()
landing_page = BeautifulSoup(page, "html.parser")

In [15]:
def li_class_resource_item(li):
    return li.has_attr("class") and "resource-item" in li["class"]


ids = [tag["data-id"] for tag in landing_page.find_all(li_class_resource_item)]

In [6]:
import re

files = list(map(lambda m: m.group(0), re.finditer(r"http://[^\"]+\.xlsx", page)))

In [8]:
import urllib

with requests.Session() as session:
    for url in files:
        r = session.get(url)
        with open(
            base / "opendata" / urllib.parse.unquote(url.split("/")[-1]), "wb"
        ) as file:
            file.write(r.content)

In [52]:
def payload(data_id, n, offset):
    return {
        "resource_id": data_id,
        "limit": n,
        "offset": offset,
    }


def get_data(data_id, n, offset):
    return requests.get(
        "https://data.civis.bz.it/api/action/datastore_search",
        params=payload(data_id, n, offset),
    ).json()

In [17]:
excels_path = base / "xlsx"
xlsx_files = list(excels_path.glob("*.xlsx"))

In [21]:
def get_meta(path):
    # ETRS89/UTM32N
    coords = pd.read_excel(path, usecols=[7], skiprows=7, nrows=3, header=None)
    meta = coords.iloc[:, 0].str.split(" ").explode().iloc[[1, 3, 4]]
    meta.index = ["x", "y", "elevation"]
    return meta

In [31]:
def read_bz_excel(path):
    original_id, station_name = path.stem.split("-", maxsplit=1)
    station_name = station_name.replace("-", " ")
    original_id = original_id.upper().strip()

    meta = get_meta(path)
    meta["original_id"] = original_id
    meta["station_name"] = station_name

    data = pd.read_excel(
        path,
        skiprows=13,
        usecols=[2, 4, 5],
        names=["date", "T_MIN", "T_MAX"],
        na_values="---",
        dtype={"T_MIN": "float", "T_MAX": "float"},
        skipfooter=1,
    )
    data = pl.from_pandas(data).with_columns(
        pl.col("date").str.to_date(format=r"%d.%m.%Y"),
        pl.lit(original_id).alias("original_id").str.to_uppercase().str.strip_chars(),
        pl.lit(station_name).alias("station_name"),
    )
    return data, meta

In [23]:
for path in xlsx_files:
    try:
        read_bz_excel(path)
    except:
        print(f"Could not open {path}")

In [32]:
all_things = [read_bz_excel(path) for path in xlsx_files]

In [ ]:
data, meta = zip(*all_things)

In [ ]:
data = pl.concat(data, how="vertical")
meta = pl.concat(
    [pl.from_pandas(pd.DataFrame(m).transpose()) for m in meta], how="vertical"
)

In [ ]:
meta = meta.with_columns(
    pl.col("x").str.to_integer(),
    pl.col("y").str.to_integer(),
    pl.col("elevation").str.to_integer(),
)

In [ ]:
meta.write_csv(base / "xlsx" / "meta.csv")

In [ ]:
data.write_parquet(base / "xlsx" / "data.parquet")